In [2]:
import pandas as pd

""" Sandobx for all things realted to Snippets extraction and generic stuff.
Not downstream things (elsewhere).
"""

' Goal: activity related to sequence of shapes (and locations, etc)\n- e.g,, reuse of activiyt.\nRequires trial-level rasters (as opposed to event-aligned rasters)\n'

In [3]:
%load_ext autoreload
%autoreload 2

from neuralmonkey.classes.session import load_mult_session_helper
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

#%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# DATE = 230623
# animal = "Pancho"
DATE = 230630
animal = "Diego"
MS = load_mult_session_helper(DATE, animal, MINIMAL_LOADING=True, spikes_version="kilosort_if_exists") 
# MS = load_mult_session_helper(DATE, animal, MINIMAL_LOADING=True, spikes_version="tdt") 

### Snippets, split by stroke index and location
MS.print_summary_sessions()
from neuralmonkey.classes.snippets import Snippets, extraction_helper

# Pick out a single session
sn = MS.SessionsList[0]


In [ ]:
# Turn on debug mode, this prunes sites and trials, so below runs faster
# sn.debug_mode_switch_to(sites=True, trials=False)
sn.debug_mode_switch_to(sites=False, trials=False)

In [ ]:
sn.sitegetter_print_summarytext_each_unit

##### Stroke snippets

In [ ]:
    
# == MODIFY THESE:
list_pre_dur = [-0.2]
list_post_dur = [0.4]
# list_pre_dur = [-0.5]
# list_post_dur = [0.5]
# which_level = "stroke_off"
which_level = "stroke"

# == IGNORE THESE:
strokes_only_keep_single = False # if True, then prunes dataset, removing trials "remove_if_multiple_behstrokes_per_taskstroke"
prune_feature_levels_min_n_trials = 1 
list_events = [] # must be empty
list_features_extraction = []
list_features_get_conjunction = []
trials_prune_just_those_including_events=False

SP = Snippets(sn,
    which_level,
    list_events,
    list_features_extraction,
    list_features_get_conjunction,
    list_pre_dur,
    list_post_dur,
    strokes_only_keep_single=strokes_only_keep_single,
    tasks_only_keep_these=None,
    prune_feature_levels_min_n_trials=prune_feature_levels_min_n_trials,
    dataset_pruned_for_trial_analysis=None,
    trials_prune_just_those_including_events=trials_prune_just_those_including_events,
    fr_which_version='sqrt',
    NEW_VERSION=True,
    SKIP_DATA_EXTRACTION=False
             )


##### Flex snippets

In [ ]:
# Main code for extracting Snippets (SP)

# == MODIFY THESE:
list_events = ["go"]
list_pre_dur = [-0.5]
list_post_dur = [0.5]

# == IGNORE THESE:
which_level = "flex"
list_features_extraction = []
list_features_get_conjunction = []
trials_prune_just_those_including_events=False

SP = Snippets(sn,
    which_level,
    list_events,
    list_features_extraction,
    list_features_get_conjunction,
    list_pre_dur,
    list_post_dur,
    strokes_only_keep_single=False,
    tasks_only_keep_these=None,
    prune_feature_levels_min_n_trials=None,
    dataset_pruned_for_trial_analysis=None,
    trials_prune_just_those_including_events=trials_prune_just_those_including_events,
    fr_which_version='sqrt',
    NEW_VERSION=True,
    SKIP_DATA_EXTRACTION=False
             )


In [ ]:
# Inspect the data. 
# Each row represents a single combination of:
# (trial, chan, event). To see that, inspect the output of 

display(SP.DfScalar)

from pythonlib.tools.pandastools import grouping_print_n_samples
grouping_print_n_samples(SP.DfScalar, ["trialcode", "chan", "event_aligned"])


##### Splitting by n taksstrokes intask

In [ ]:
D = SP.datasetbeh_extract_dataset()
D.extract_beh_features(["num_strokes_task"])
SP.datasetbeh_append_column("FEAT_num_strokes_task", D)
# Conjunction of stroke index and num strokes in task.
from pythonlib.tools.pandastools import append_col_with_grp_index
SP.DfScalar = append_col_with_grp_index(SP.DfScalar, ["FEAT_num_strokes_task", "stroke_index"], "nstk_stkidx", False)
SP.DfScalar["nstk_stkidx"].value_counts()

SP.DfScalar_OutlierRows = append_col_with_grp_index(SP.DfScalar_OutlierRows, ["FEAT_num_strokes_task", "stroke_index"], "nstk_stkidx", False)
SP.DfScalar_OutlierRows["nstk_stkidx"].value_counts()



##### Plotting functions for Snippets

In [ ]:
SAVEDIR = "/gorilla1/analyses/recordings/main/shape_vs_loc_index_PIG"
import os
savedir = f"{SAVEDIR}/{animal}/{DATE}"
os.makedirs(savedir, exist_ok=True)
print(savedir)

In [ ]:
if DEBUG:
    # if you debug, then you have small n trials, so lower the threshold for keeping data.
    print(SP.ParamsGlobals)1
    SP.globals_update(globals_nmin=1)


In [ ]:
chan = SP.Sites[0]
SP.plotgood_rasters(chan)

In [ ]:
SP.plotgood_rasters_smfr_combined(chan)

In [ ]:
sn.sitegetter_print_summarytext_each_unit()

In [ ]:
chan = SP.Sites[1]
for PLOT_VER in ["smfr", "raster"]:
    SP.plotgood_smfr_each_level_subplot_grid_by_vars(chan, "shape_oriented", 
                                                     "gridloc", "stroke_index",
                                                     PLOT_VER=PLOT_VER);

In [ ]:
# Rasters and smoothed FR, split by event

# SP.datasetbeh_append_column("epoch", sn.Datasetbeh) # assign epoch to SP

chan = SP.Sites[1]
SP.plotgood_rasters_smfr_each_level_combined(chan, var="shape_oriented", vars_others=["gridloc", "stroke_index"]);
SP.plotgood_rasters_smfr_each_level_combined(chan, var="shape_oriented", vars_others=["gridloc"]);
SP.plotgood_rasters_smfr_each_level_combined(chan, var="shape_oriented", vars_others=["stroke_index"]);


In [ ]:
# Show heatmap, conjucntion of other variables (how balanced is it).
from pythonlib.tools.pandastools import convert_to_2d_dataframe, grouping_plot_n_samples_conjunction_heatmap
# 
# convert_to_2d_dataframe(SP.DfScalar, "")

fig = grouping_plot_n_samples_conjunction_heatmap(SP.DfScalar, "shape_oriented", "gridloc", ["stroke_index"])

In [ ]:
chan = SP.Sites[6]
PLOT_VER = "smfr"
SP.plotgood_smfr_each_level_subplot_grid_by_vars(chan, "shape_oriented", 
                                                 "gridloc", "nstk_stkidx",
                                                 PLOT_VER=PLOT_VER);

In [ ]:
chan = SP.Sites[1]

In [ ]:
chan = SP.Sites[1]

In [ ]:
SP.plotgood_rasters_smfr_each_level_combined(chan, var="shape_oriented", vars_others=["nstk_stkidx"]);


##### Time warping, so that can look at offset and onset

In [ ]:
SP.DfScalar[:2]

### Computing modulation of activity by certain variables (site by site)

For each site, compute how strongly it is modulated by a specific variable, computed 
separatedly for each event.

In [ ]:
SP.DfScalar[:2]

In [ ]:
# First, append the variables that you wish to analyze
SP.datasetbeh_append_column("epoch", sn.Datasetbeh) # assign epoch to SP


In [ ]:
if DEBUG:
    # if you debug, then you have small n trials, so lower the threshold for keeping data.
    print(SP.ParamsGlobals)
    SP.globals_update(globals_nmin=2)


In [ ]:
# Run this to compute modulation
DF_VAR, DF_FR, DF_FR_LEVELS, list_eventwindow_event = SP.modulationgood_compute_wrapper("epoch", 
                                                                                        DEBUG_CONJUNCTIONS=False, 
                                                                                        THINGS_TO_EXTRACT=["anova"])


In [ ]:
# Plot results and save
sdir_base = "/tmp"
SP.modulationgood_plot_WRAPPER(DF_VAR, DF_FR, DF_FR_LEVELS, list_eventwindow_event, 
                               var="epoch", vars_conjuction=["dummy_var"],
                               sdir_base=sdir_base, N_WAYS=1)

### Modulation by time (relative to aligned events)

For each site x event, compute a scalar value indiicating how much its actiivyt is modulated
with respect to this event. This moudlation can be increase in fr, decrease, or any
activity pattern that is consistent across trials.

In [ ]:
# Extract modulation, for each site x event

In [ ]:
from neuralmonkey.analyses.event_temporal_modulation_v2 import preprocess_and_extract
from neuralmonkey.analyses.event_temporal_modulation import plot_overview

df_modtime = preprocess_and_extract(SP)


In [ ]:
# Plot overview of modulation

In [ ]:
from neuralmonkey.analyses.event_temporal_modulation_v2 import plot_overview
SAVEDIR = "/tmp"
##################### PLOTS
plot_overview(df_modtime, SP, SAVEDIR)

## Sliding decoder

In [ ]:
# 1) Separate decoders aligned to stroke onset. (grouping all stroke).
# 2) 


In [ ]:
Look at 220713_prims_state_space

# ### Scratch plots

In [ ]:
PA.plotNeurHeat(1)
PAnorm.plotNeurHeat(1)

In [ ]:
trial = 10
PA.plotNeurTimecourse(trial)
PAnorm.plotNeurTimecourse(trial)

In [ ]:
X = np.mean(PAnorm.X, axis=1, keepdims=False)
import seaborn as sns
sns.heatmap(X)

In [ ]:
SP.plotgood_rasters

In [ ]:
# pull out pa for a specific chan
chan = SP.Sites[4]

In [ ]:
chan = 50

In [ ]:
PAnorm.Chans

In [ ]:
SP.SN.sitegetter_print_summarytext_each_unit()